<a href="https://colab.research.google.com/github/SimranAnand2/Autograding-handwritten-mathematical-worksheets/blob/master/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch>=1.9
!pip install fastai>=2.7
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install torchvision
!pip install tqdm
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
DEBUG = False # set this flag to True to use a small subset of data for testing
     

from fastai.vision.all import *
import params

import wandb
     
# We have defined some global configuration parameters in the params.py file. ENTITY should correspond to your W&B Team name if you work in a team, replace it with None if you work individually.

# In the section below, we will use untar_data function from fastai to download and unzip our datasets.


URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'
     

path = Path(untar_data(URL, force_download=True))
     

path.ls()
     
# Here we define several functions to help us process the data and upload it as a Table to W&B.


def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table
     
# We will start a new W&B run and put everything into a raw Artifact.


run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")
     

raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')
     
# Let's add the images and label masks.


raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')
     
# Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table.


image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]
     

table = _create_table(image_files, params.BDD_CLASSES)
     
# Finally, we will add the Table to our Artifact, log it to W&B and finish our run.


raw_data_at.add(table, "eda_table")
     

run.log_artifact(raw_data_at)
run.finish()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/images)... Done. 0.9s
wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/labels)... Done. 0.5s


In [3]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")
     
# Let's use artifact we previously saved to W&B (we're storing artifact names and other global parameters in params).


raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())
     

path.ls()
     
# To split data between training, testing and validation, we need file names, groups (derived from the file name) and target (here we use our rare class bicycle for stratification). We previously saved these columns to EDA table, so let's retrieve it from the table now.


fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]
     

orig_eda_table = raw_data_at.get("eda_table")
     

y = orig_eda_table.get_column('bicycle')
     
# Now we will split the data into train (80%), validation (10%) and test (10%) sets. As we do that, we need to be careful to:

# avoid leakage: for that reason we are grouping data according to video identifier (we want to make sure our model can generalize to new cars or video frames)
# handle the label imbalance: for that reason we stratify data with our target column
# We will use sklearn's StratifiedGroupKFold to split the data into 10 folds and assign 1 fold for test, 1 for validation and the rest for training.


df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1
     
import sklearn 
from sklearn.model_selection import StratifiedGroupKFold 
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i
     

df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()
     

df.to_csv('data_split.csv', index=False)
     
# We will now create a new artifact and add our data there.


processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")
     

processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)
     
# Finally, the split information may be relevant for our analyses - rather than uploading images again, we will save the split information to a new table and join it with EDA table we created previously.


data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])
     

join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")
     
# Let's add it to our artifact, log it and finish our run.


processed_data_at.add(join_table, "eda_table_data_split")
     

run.log_artifact(processed_data_at)
run.finish()

wandb: Currently logged in as: simrananand. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:46.5
wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:0.4
wandb: Adding directory to artifact (./artifacts/bdd_simple_1k:v1)... Done. 5.1s


In [4]:
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

from fastai.callback.wandb import *
     
# Again, we're importing some global configuration parameters from params.py file. We have also defined some helper functions in utils.py - for example metrics we will track during our experiments.

# Let's now create a train_config that we'll pass to W&B run to control training hyperparameters.


train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
)
     
# We are setting seed for reproducibility.


set_seed(train_config.seed, reproducible=True)
     

run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=train_config)
     
# As usual, we will use W&B Artifacts to track the lineage of our models.


processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')
     
# We will not use the hold out dataset stage at this moment. is_valid column will tell our trainer how we want to split data between training and validation.


df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'
     

def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"
     
# We will use fastai's DataBlock API to feed data into model training and validation.


# assign paths
df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
df["label_fname"] = [label_func(f) for f in df.image_fname.values]
     

def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)
     
# We are using wandb.config to track our training hyperparameters.


config = wandb.config
     

dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)
     
# We will use intersection over union metrics: mean across all classes (MIOU) and IOU for each class separately. Our model will be a unet based on pretrained resnet18 backbone.


metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
           TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)
     
# In fastai we already have a callback that integrates tightly with W&B, we only need to pass the WandbCallback to the learner and we are ready to go. The callback will log all the useful variables for us. For example, whatever metric we pass to the learner will be tracked by the callback.


callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]
     
# Let's train our model!


learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)
     
# We will log a table with model predictions and ground truth to W&B, so that we can do error analysis in the W&B dashboard.


samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({"pred_table":table})
     
# We are reloading the model from the best checkpoint at the end and saving it. To make sure we track the final metrics correctly, we will validate the model again and save the final loss and metrics to wandb.summary.


scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v
     

wandb.finish()
     

wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:3.8
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.508330,0.399609,0.283337,0.853196,0.629339,0.000000,0.000000,0.000000,0.500827,0.000000,00:53
1,0.428257,0.347966,0.317667,0.865419,0.770114,0.000000,0.000000,0.000000,0.588139,0.000000,00:48
2,0.358932,0.290348,0.331854,0.890223,0.801375,0.000000,0.000000,0.000000,0.631383,0.000000,00:50
3,0.315577,0.266757,0.344216,0.902056,0.818195,0.000000,0.000000,0.000000,0.689262,0.000000,00:50
4,0.280642,0.251853,0.351343,0.910763,0.822296,0.000000,0.000000,0.000000,0.726345,0.000000,00:49
5,0.243859,0.242683,0.352813,0.910420,0.819074,0.000000,0.000000,0.000000,0.740195,0.000000,00:49
6,0.231552,0.242748,0.357629,0.917590,0.837937,0.000000,0.000000,0.000000,0.747877,0.000000,00:49
7,0.203880,0.226904,0.362983,0.920724,0.843411,0.020426,0.000000,0.000000,0.756323,0.000000,00:49
8,0.192214,0.227138,0.363634,0.920881,0.844893,0.021191,0.000000,0.000000,0.758473,0.000000,00:50
9,0.180399,0.226384,0.367253,0.920890,0.845154,0.047896,0.000000,0.000000,0.756829,0.000000,00:49


Better model found at epoch 0 with miou value: 0.2833373843877895.
Better model found at epoch 1 with miou value: 0.31766730132092835.
Better model found at epoch 2 with miou value: 0.3318544512035495.
Better model found at epoch 3 with miou value: 0.3442160105312699.
Better model found at epoch 4 with miou value: 0.3513432452889295.
Better model found at epoch 5 with miou value: 0.3528125653804222.
Better model found at epoch 6 with miou value: 0.3576291620521715.
Better model found at epoch 7 with miou value: 0.3629833850217671.
Better model found at epoch 8 with miou value: 0.36363394618301065.
Better model found at epoch 9 with miou value: 0.3672525553783344.


background_iou,▁▂▅▆▇▇████
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_2,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
miou,▁▄▅▆▇▇▇███
mom_0,██▇▆▅▄▃▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████
